In [4]:
! pip install transformers

In [5]:
import pandas as pd
import numpy as np
import tensorflow as tf
import glob
from bert_train import train_bert



## Training on twitter dataset

In [6]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import re
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import os

#### Initialize bert classification model

In [7]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures

model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased", 
                                                        trainable=True, 
                                                        num_labels=3)
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [9]:
df = pd.read_csv("data/twitter.csv")
df = df.drop(columns=['Unnamed: 0'])
df['tweet'] = df['tweet'].str.strip()
df.count()

tweet    24472
label    24472
dtype: int64

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24472 entries, 0 to 24471
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tweet   24472 non-null  object
 1   label   24472 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 382.5+ KB


In [11]:
def get_dataset(df, seed, test_size):
    return train_test_split(df, test_size=test_size, random_state=seed, shuffle=True)

In [12]:
train, test = get_dataset(df, 11, 0.2)

In [13]:
train = train.append(test)

In [14]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24472 entries, 11983 to 24279
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tweet   24472 non-null  object
 1   label   24472 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 573.6+ KB


In [16]:
train.columns = ['DATA_COLUMN', 'LABEL_COLUMN']
test.columns = ['DATA_COLUMN', 'LABEL_COLUMN']

In [17]:
def convert_data_to_examples(train, test, DATA_COLUMN, LABEL_COLUMN): 
  train_InputExamples = train.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)

  validation_InputExamples = test.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)
  
  return train_InputExamples, validation_InputExamples

  train_InputExamples, validation_InputExamples = convert_data_to_examples(train, 
                                                                           test, 
                                                                           'DATA_COLUMN', 
                                                                           'LABEL_COLUMN')
  
def convert_examples_to_tf_dataset(examples, tokenizer, max_length=128):
    features = [] # -> will hold InputFeatures to be converted later

    for e in examples:
        # Documentation is really strong for this method, so please take a look at it
        input_dict = tokenizer.encode_plus(
            e.text_a,
            add_special_tokens=True,
            max_length=max_length, # truncates if len(s) > max_length
            return_token_type_ids=True,
            return_attention_mask=True,
            pad_to_max_length=True, # pads to the right by default # CHECK THIS for pad_to_max_length
            truncation=True
        )

        input_ids, token_type_ids, attention_mask = (input_dict["input_ids"],
            input_dict["token_type_ids"], input_dict['attention_mask'])

        features.append(
            InputFeatures(
                input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, label=e.label
            )
        )

    def gen():
        for f in features:
            yield (
                {
                    "input_ids": f.input_ids,
                    "attention_mask": f.attention_mask,
                    "token_type_ids": f.token_type_ids,
                },
                f.label,
            )

    return tf.data.Dataset.from_generator(
        gen,
        ({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.int64),
        (
            {
                "input_ids": tf.TensorShape([None]),
                "attention_mask": tf.TensorShape([None]),
                "token_type_ids": tf.TensorShape([None]),
            },
            tf.TensorShape([]),
        ),
    )


DATA_COLUMN = 'DATA_COLUMN'
LABEL_COLUMN = 'LABEL_COLUMN'

In [18]:
train_InputExamples, validation_InputExamples = convert_data_to_examples(train, test, DATA_COLUMN, LABEL_COLUMN)

train_data = convert_examples_to_tf_dataset(list(train_InputExamples), tokenizer)
train_data = train_data.batch(32)

validation_data = convert_examples_to_tf_dataset(list(validation_InputExamples), tokenizer)
validation_data = validation_data.batch(32)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2110: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [20]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])



In [21]:
hist = model.fit(train_data, epochs=6, validation_data=validation_data, callbacks=[cp_callback])

Epoch 1/6
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autogr

In [22]:
preds = model.predict(validation_data)

In [23]:
cr = classification_report(test['LABEL_COLUMN'],np.argmax(preds[0],axis=1),output_dict=True)

In [24]:
pd.DataFrame(cr).transpose().to_csv('classification_reports/classification_mat_bert_cv_6epochs_tweet_finetune.csv')



---



---



---



---



In [25]:
from transformers import BertTokenizer, TFBertForSequenceClassification

model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                        trainable=True,
                                                        num_labels=3)
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Cross validation by loading above training set

In [26]:
# perform cross folding
def load_movies_to_df(path):
    df_movies = []

    for filename in glob.glob(path + '*.csv'):
        df_movies.append(pd.read_csv(filename))

    return df_movies

In [27]:
df_movies = load_movies_to_df('movies/for_training/')
classification_report = []
df_main = pd.DataFrame()

In [28]:
for i in range(len(df_movies)):
    df_train = pd.concat(df_movies[0:i] + df_movies[i + 1:])
    df_test = df_movies[i]

    train_movies = df_train['movie_name'].unique()
    test_movie = df_test['movie_name'].unique()
    print(','.join(train_movies))
    print(test_movie[0])
    classification_report.append(train_bert(df_train, df_test))
    # classification_report.append(x)
    print('Train movies: ', str(','.join(train_movies)))
    print('Test movie: ', str(test_movie[0]))
    print('Classification report: \n', classification_report[i])
    print('------------------------------------------------')

    df_cr = pd.DataFrame(classification_report[i]).transpose()
    df_cr['movie_train'] =  str(','.join(train_movies))
    df_cr['movie_test'] = str(test_movie[0])
    df_cr.to_csv('classification_reports/'+str(test_movie[0])+'.csv')
    df_main = df_main.append(df_cr)

South_Park,Django_Unchained,Pulp_Fiction,AmerricanHistoryX,TheWolfofWallStreet
BlacKkKlansman


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2110: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Epoch 1/5
283/283 [==============================] - 244s 796ms/step - loss: 0.2415 - accuracy: 0.9350
Epoch 2/5
283/283 [==============================] - 225s 794ms/step - loss: 0.1185 - accuracy: 0.9596
Epoch 3/5
283/283 [==============================] - 225s 795ms/step - loss: 0.0676 - accuracy: 0.9778
Epoch 4/5
283/283 [==============================] - 225s 795ms/step - loss: 0.0372 - accuracy: 0.9901
Epoch 5/5
283/283 [==============================] - 225s 795ms/step - loss: 0.0259 - accuracy: 0.9925
predicting
Train movies:  South_Park,Django_Unchained,Pulp_Fiction,AmerricanHistoryX,TheWolfofWallStreet
Test movie:  BlacKkKlansman
Classification report: 
 {'0': {'precision': 0.9574036511156186, 'recall': 0.9685362517099864, 'f1-score': 0.9629377762665761, 'support': 1462}, '1': {'precision': 0.5, 'recall': 0.5773195876288659, 'f1-score': 0.5358851674641149, 'support': 97}, '2': {'precision': 0.8888888888888888, 'recall': 0.5581395348837209, 'f1-score': 0.6857142857142857, 'sup

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/5
302/302 [==============================] - 259s 795ms/step - loss: 0.2779 - accuracy: 0.9337
Epoch 2/5
302/302 [==============================] - 240s 793ms/step - loss: 0.1528 - accuracy: 0.9560
Epoch 3/5
302/302 [==============================] - 240s 794ms/step - loss: 0.1181 - accuracy: 0.9569
Epoch 4/5
302/302 [==============================] - 240s 793ms/step - loss: 0.0756 - accuracy: 0.9694
Epoch 5/5
302/302 [==============================] - 240s 794ms/step - loss: 0.0564 - accuracy: 0.9787
predicting
Train movies:  BlacKkKlansman,Django_Unchained,Pulp_Fiction,AmerricanHistoryX,TheWolfofWallStreet
Test movie:  South_Park
Classification report: 
 {'0': {'precision': 0.9481236203090507, 'recall': 0.9651685393258427, 'f1-score': 0.9565701559020043, 'support': 890}, '1': {'precision': 0.7611940298507462, 'recall': 0.7034482758620689, 'f1-score': 0.7311827956989247, 'support': 145}, '2': {'precision': 0.3333333333333333, 'recall': 0.18181818181818182, 'f1-score': 0.235294

...

Epoch 1/5
284/284 [==============================] - 244s 796ms/step - loss: 0.2994 - accuracy: 0.9263
Epoch 2/5
284/284 [==============================] - 225s 794ms/step - loss: 0.1627 - accuracy: 0.9426
Epoch 3/5
284/284 [==============================] - 226s 795ms/step - loss: 0.1145 - accuracy: 0.9577
Epoch 4/5
284/284 [==============================] - 226s 794ms/step - loss: 0.0669 - accuracy: 0.9758
Epoch 5/5
284/284 [==============================] - 226s 795ms/step - loss: 0.0396 - accuracy: 0.9872
predicting
Train movies:  BlacKkKlansman,South_Park,Django_Unchained,AmerricanHistoryX,TheWolfofWallStreet
Test movie:  Pulp_Fiction
Classification report: 
 {'0': {'precision': 0.9725818735719726, 'recall': 0.9579894973743436, 'f1-score': 0.9652305366591082, 'support': 1333}, '1': {'precision': 0.7964912280701755, 'recall': 0.8566037735849057, 'f1-score': 0.8254545454545454, 'support': 265}, '2': {'precision': 0.8333333333333334, 'recall': 0.8333333333333334, 'f1-score': 0.833333

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/5
286/286 [==============================] - 246s 793ms/step - loss: 0.2923 - accuracy: 0.9268
Epoch 2/5
286/286 [==============================] - 227s 793ms/step - loss: 0.1654 - accuracy: 0.9523
Epoch 3/5
286/286 [==============================] - 227s 794ms/step - loss: 0.1158 - accuracy: 0.9598
Epoch 4/5
286/286 [==============================] - 227s 793ms/step - loss: 0.0771 - accuracy: 0.9732
Epoch 5/5
286/286 [==============================] - 227s 794ms/step - loss: 0.0512 - accuracy: 0.9820
predicting
Train movies:  BlacKkKlansman,South_Park,Django_Unchained,Pulp_Fiction,TheWolfofWallStreet
Test movie:  AmerricanHistoryX
Classification report: 
 {'0': {'precision': 0.9628506444275967, 'recall': 0.9739263803680982, 'f1-score': 0.9683568433091879, 'support': 1304}, '1': {'precision': 0.7741935483870968, 'recall': 0.8115942028985508, 'f1-score': 0.7924528301886792, 'support': 207}, '2': {'precision': 0.9655172413793104, 'recall': 0.5185185185185185, 'f1-score': 0.674698

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/5
239/239 [==============================] - 209s 795ms/step - loss: 0.3183 - accuracy: 0.9213
Epoch 2/5
239/239 [==============================] - 190s 794ms/step - loss: 0.1645 - accuracy: 0.9490
Epoch 3/5
239/239 [==============================] - 190s 794ms/step - loss: 0.1176 - accuracy: 0.9577
Epoch 4/5
239/239 [==============================] - 190s 794ms/step - loss: 0.0918 - accuracy: 0.9636
Epoch 5/5
239/239 [==============================] - 190s 795ms/step - loss: 0.0659 - accuracy: 0.9760
predicting
Train movies:  BlacKkKlansman,South_Park,Django_Unchained,Pulp_Fiction,AmerricanHistoryX
Test movie:  TheWolfofWallStreet
Classification report: 
 {'0': {'precision': 0.988491574188245, 'recall': 0.9721099434114794, 'f1-score': 0.9802323211738333, 'support': 2474}, '1': {'precision': 0.9053156146179402, 'recall': 0.9284497444633731, 'f1-score': 0.9167367535744324, 'support': 587}, '2': {'precision': 0.07142857142857142, 'recall': 1.0, 'f1-score': 0.13333333333333333, 's

In [29]:
df_main.to_csv('cross_fold_results_tweet_finetune_6epochs.csv')

In [30]:
print(df_main)

              precision  ...           movie_test
0              0.957404  ...       BlacKkKlansman
1              0.500000  ...       BlacKkKlansman
2              0.888889  ...       BlacKkKlansman
accuracy       0.924012  ...       BlacKkKlansman
macro avg      0.782098  ...       BlacKkKlansman
weighted avg   0.926850  ...       BlacKkKlansman
0              0.948124  ...           South_Park
1              0.761194  ...           South_Park
2              0.333333  ...           South_Park
accuracy       0.920650  ...           South_Park
macro avg      0.680884  ...           South_Park
weighted avg   0.915746  ...           South_Park
0              0.979474  ...     Django_Unchained
1              0.701299  ...     Django_Unchained
2              0.981982  ...     Django_Unchained
accuracy       0.967373  ...     Django_Unchained
macro avg      0.887585  ...     Django_Unchained
weighted avg   0.967063  ...     Django_Unchained
0              0.972582  ...         Pulp_Fiction


In [31]:
len(classification_report[0])

6

In [32]:
df_main.head()

,precision,recall,f1-score,support,movie_train,movie_test
0,0.957404,0.968536,0.962938,1462.000000,"South_Park,Django_Unchained,Pulp_Fiction,Amerr...",BlacKkKlansman
1,0.500000,0.577320,0.535885,97.000000,"South_Park,Django_Unchained,Pulp_Fiction,Amerr...",BlacKkKlansman
2,0.888889,0.558140,0.685714,86.000000,"South_Park,Django_Unchained,Pulp_Fiction,Amerr...",BlacKkKlansman
accuracy,0.924012,0.924012,0.924012,0.924012,"South_Park,Django_Unchained,Pulp_Fiction,Amerr...",BlacKkKlansman
macro avg,0.782098,0.701332,0.728179,1645.000000,"South_Park,Django_Unchained,Pulp_Fiction,Amerr...",BlacKkKlansman


In [33]:
def get_precision_recall_f1(category, result_df):
    precision = result_df[result_df.label==category].precision.mean()
    recall = result_df[result_df.label==category].recall.mean()
    f1 = result_df[result_df.label==category]['f1-score'].mean()
    
    return {'label': category, 'precision': precision, 'recall': recall, 'f1': f1}

In [34]:
df_cv= pd.read_csv('cross_fold_results_tweet_finetune_6epochs.csv')

In [35]:
df_cv = df_cv.rename(columns={'Unnamed: 0': 'label', 'b': 'Y'})
df_cv.head()

,label,precision,recall,f1-score,support,movie_train,movie_test
0,0,0.957404,0.968536,0.962938,1462.000000,"South_Park,Django_Unchained,Pulp_Fiction,Amerr...",BlacKkKlansman
1,1,0.500000,0.577320,0.535885,97.000000,"South_Park,Django_Unchained,Pulp_Fiction,Amerr...",BlacKkKlansman
2,2,0.888889,0.558140,0.685714,86.000000,"South_Park,Django_Unchained,Pulp_Fiction,Amerr...",BlacKkKlansman
3,accuracy,0.924012,0.924012,0.924012,0.924012,"South_Park,Django_Unchained,Pulp_Fiction,Amerr...",BlacKkKlansman
4,macro avg,0.782098,0.701332,0.728179,1645.000000,"South_Park,Django_Unchained,Pulp_Fiction,Amerr...",BlacKkKlansman


In [36]:
normal_dict = get_precision_recall_f1('0', df_cv)
offensive_dict = get_precision_recall_f1('1',df_cv)
hate_dict = get_precision_recall_f1('2',df_cv)

In [37]:
df_result = pd.DataFrame([normal_dict, offensive_dict, hate_dict])
df_result

,label,precision,recall,f1
0,0,0.968154,0.970376,0.969220
1,1,0.739749,0.760160,0.749003
2,2,0.679081,0.670572,0.586419


In [38]:
for cr in classification_report:
  print(cr)

{'0': {'precision': 0.9574036511156186, 'recall': 0.9685362517099864, 'f1-score': 0.9629377762665761, 'support': 1462}, '1': {'precision': 0.5, 'recall': 0.5773195876288659, 'f1-score': 0.5358851674641149, 'support': 97}, '2': {'precision': 0.8888888888888888, 'recall': 0.5581395348837209, 'f1-score': 0.6857142857142857, 'support': 86}, 'accuracy': 0.9240121580547113, 'macro avg': {'precision': 0.7820975133348358, 'recall': 0.7013317914075244, 'f1-score': 0.7281790764816588, 'support': 1645}, 'weighted avg': {'precision': 0.9268502020519628, 'recall': 0.9240121580547113, 'f1-score': 0.9232628077308096, 'support': 1645}}
{'0': {'precision': 0.9481236203090507, 'recall': 0.9651685393258427, 'f1-score': 0.9565701559020043, 'support': 890}, '1': {'precision': 0.7611940298507462, 'recall': 0.7034482758620689, 'f1-score': 0.7311827956989247, 'support': 145}, '2': {'precision': 0.3333333333333333, 'recall': 0.18181818181818182, 'f1-score': 0.23529411764705885, 'support': 11}, 'accuracy': 0.92